In [44]:
import pandas as pd

In [45]:
df=pd.read_json('loan_data.json')

In [46]:
df

,Application_ID,Gender,Married,Dependents,Education,Self_Employed,Credit_History,Property_Area,Income,Application_Status
0,LP001002,Male,No,0,Graduate,No,1,Urban,medium,Y
1,LP001003,Male,Yes,1,Graduate,No,1,Rural,medium,N
2,LP001005,Male,Yes,0,Graduate,Yes,1,Urban,low,Y
3,LP001006,Male,Yes,0,Not Graduate,No,1,Urban,low,Y
4,LP001008,Male,No,0,Graduate,No,1,Urban,medium,Y
...,...,...,...,...,...,...,...,...,...,...
506,LP002978,Female,No,0,Graduate,No,1,Rural,low,Y
507,LP002979,Male,Yes,3+,Graduate,No,1,Rural,medium,Y
508,LP002983,Male,Yes,1,Graduate,No,1,Urban,medium,Y
509,LP002984,Male,Yes,2,Graduate,No,1,Urban,medium,Y


In [47]:
un_depend=df['Dependents'].value_counts()

In [48]:
un_depend

0     294
2      88
1      85
3+     44
Name: Dependents, dtype: int64

In [49]:
x=['0','1','2','3+']
print("Percentage of applicants in following dependents: ")
for i,j in zip(x,un_depend):
    print("{} : {}".format(i,j/sum(un_depend)*100))
    

Percentage of applicants in following dependents: 
0 : 57.534246575342465
1 : 17.22113502935421
2 : 16.634050880626223
3+ : 8.610567514677104


In [50]:
c=0
d=0
for i,j in zip(df['Self_Employed'],df['Application_Status']):
    if i=='Yes' and j=='Y':
        c=c+1
    if i=='Yes':
        d=d+1
print("Percentage of applications approved for self-employed applicants : {}".format(c/d*100))

Percentage of applications approved for self-employed applicants : 65.71428571428571


In [51]:
c=0
d=0
for i,j,k in zip(df['Married'],df['Gender'],df['Application_Status']):
    if i=='Yes' and j=='Male'and k=='N':
        c=c+1
    if i=='Yes' and j=='Male':
        d=d+1
print("% of rejections for married male applicants : {}".format(c/d*100))

% of rejections for married male applicants : 28.431372549019606


In [52]:
sel=df['Property_Area'].value_counts()

In [53]:
sel

Semiurban    197
Urban        165
Rural        149
Name: Property_Area, dtype: int64

In [56]:
P_Area = df.groupby('Property_Area')
print(P_Area)

p_type = []
Approval_ratio = []
i = 0
for a,j in P_Area:
    p_type.append(a)
    prop = j
    
    arr = prop['Application_Status'].value_counts()     
    print(arr) # 0th index: 'Y', 1st index: 'N'
    Approval_ratio.append(arr[0]/sum(arr)*100)
    print("Approval ratio for {}: {}".format(p_type[i], Approval_ratio[i]))
    print()
    i += 1
print()
print("Maximum approval ratio:- {}\nType:- {}".format(max(Approval_ratio), p_type[Approval_ratio.index(max(Approval_ratio))]))

Y    90
N    59
Name: Application_Status, dtype: int64
Approval ratio for Rural: 60.40268456375839

Y    153
N     44
Name: Application_Status, dtype: int64
Approval ratio for Semiurban: 77.66497461928934

Y    104
N     61
Name: Application_Status, dtype: int64
Approval ratio for Urban: 63.030303030303024


Maximum approval ratio:- 77.66497461928934
Type:- Semiurban


In [69]:
df1 = df.drop(['Application_ID', 'Gender', 'Married', 'Education','Self_Employed', 'Credit_History', 'Property_Area','Application_Status'],axis=1) 
df1.replace(to_replace=["3+"],value="3",inplace=True)
df1['Dependents'] = df1['Dependents'].astype(int)
Income = df1.groupby('Income')

for a,j in Income:
    print(a)
    print(sum(j['Dependents']))
    print(j['Dependents'].count())
    print("Average dependents of {} income group: {}".format(a,sum(j['Dependents'])/j['Dependents'].count()))
    print()

high
40
45
Average dependents of high income group: 0.8888888888888888

low
175
273
Average dependents of low income group: 0.6410256410256411

medium
178
193
Average dependents of medium income group: 0.9222797927461139



In [88]:
df.columns

Index(['Application_ID', 'Gender', 'Married', 'Dependents', 'Education',
       'Self_Employed', 'Credit_History', 'Property_Area', 'Income',
       'Application_Status'],
      dtype='object')

In [89]:
df.replace(to_replace=["3+"],value="3",inplace=True)


In [90]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(df, test_size=0.2, random_state=42)


In [91]:
X_train = train_set.drop(['Application_ID','Application_Status'],axis=1)
y_train = train_set['Application_Status'].copy()

In [92]:
y_train

208    Y
204    Y
301    Y
390    N
416    N
      ..
106    Y
270    Y
348    N
435    Y
102    Y
Name: Application_Status, Length: 408, dtype: object

In [93]:
X_train = pd.get_dummies(X_train, columns = ['Gender','Married','Education','Self_Employed','Property_Area','Income'])

In [94]:
X_train.columns


Index(['Dependents', 'Credit_History', 'Gender_Female', 'Gender_Male',
       'Married_No', 'Married_Yes', 'Education_Graduate',
       'Education_Not Graduate', 'Self_Employed_No', 'Self_Employed_Yes',
       'Property_Area_Rural', 'Property_Area_Semiurban', 'Property_Area_Urban',
       'Income_high', 'Income_low', 'Income_medium'],
      dtype='object')

In [95]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler,Normalizer
my_pipeline = Pipeline([
    ('std_scaler', StandardScaler())
])

In [96]:
tr_X_train = my_pipeline.fit_transform(X_train)

In [97]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
labels = le.fit_transform(y_train)

In [98]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(X_train,labels)

RandomForestClassifier()

In [99]:
from sklearn.metrics import mean_squared_error, accuracy_score
import numpy as np
y_pred = model.predict(X_train)
mse = mean_squared_error(labels, y_pred)
rmse = np.sqrt(mse)

In [100]:
rmse

0.34655164004183603

In [101]:
ac = accuracy_score(labels, y_pred)
ac

0.8799019607843137

In [115]:
X_test = test_set.drop(['Application_ID','Application_Status'],axis=1)
y_test = test_set['Application_Status'].copy()

In [116]:
X_test = pd.get_dummies(X_test, columns = ['Gender','Married','Education','Self_Employed','Property_Area','Income'])

In [117]:
labels_test = le.fit_transform(y_test)

In [118]:
tr_X_test = my_pipeline.fit_transform(X_test)

In [119]:
final_predictions = model.predict(tr_X_test)
final_mse = mean_squared_error(labels_test, final_predictions)
final_rmse = np.sqrt(final_mse)

C:\Users\rishu\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [120]:
final_rmse

0.8302532073708084

In [ ]:
import seaborn as se
from tensorflow.math import confusion_matrix
import matplotlib.pyplot as plt


y_classes = final_predictions
cm = confusion_matrix(labels_test,y_classes)

plt.figure(figsize=(15,10))
se.heatmap(cm, annot = True)
plt.show()